In [1]:
import pandas as pd
import numpy as np
import os

## Louisiana 2024 General Election Turnout 

In [2]:
holder = pd.read_excel("./raw-from-source/2024_1105_par.xls", sheet_name = None, skiprows = 8)

WARNING *** file size (3874752) not 512 + multiple of sector size (512)


In [3]:
sheets = list(holder.keys())
counties = []

for key in sheets:
    gen24_turnout = pd.read_excel("./raw-from-source/2024_1105_par.xls", sheet_name = key, skiprows = 8)
    county = gen24_turnout.loc[0,"Unnamed: 1"]
    gen24_turnout.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1, inplace = True)
    gen24_turnout.drop(0, axis = 0 , inplace = True)
    gen24_turnout.rename(columns = {"Unnamed: 1": "Precinct"}, inplace = True)

    totals = gen24_turnout[0::3].copy()
    totals.reset_index(inplace = True, drop = True)

    voting = gen24_turnout[1::3].copy()
    voting.reset_index(inplace = True, drop = True)

    pct = gen24_turnout[2::3].copy()
    pct.reset_index(inplace = True, drop = True)

    pct.drop(["Precinct"], axis = 1, inplace = True)
    voting.drop(["Precinct"], axis = 1, inplace = True)

    voting.columns = [i.replace(".1","_Dem").replace(".2","_Rep").replace(".3","_Oth") + "_Voted" for i in list(voting.columns)]
    totals.columns = [i.replace(".1","_Dem").replace(".2","_Rep").replace(".3","_Oth") + "_Reg" if i != "Precinct"  else i for i in list(totals.columns)]
    pct.columns = [i.replace(".1","_Dem").replace(".2","_Rep").replace(".3","_Oth") + "_Pct" for i in list(pct.columns)]

    gen24_turnout = pd.concat([totals,voting,pct], axis = 1)

    gen24_turnout.insert(1, "Parish", county)
    gen24_turnout.loc[0,"Precinct"] = "Parishwide"
    gen24_turnout["Precinct"] = np.where(gen24_turnout["Precinct"]!= "Parishwide",gen24_turnout["Precinct"].astype(str).str[7:],"Parishwide")
    counties.append(gen24_turnout)

WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector size (512)
WARNING *** file size (3874752) not 512 + multiple of sector siz

In [4]:
statewide = pd.concat(counties)

In [5]:
statewide["join_col"] = statewide["Parish"]+"-:-"+statewide["Precinct"].str.replace("W ","").str.replace(" P "," ")
election_results = pd.read_csv("./raw-from-source/la_2024_gen_prec/la_2024_gen_prec.csv")
election_results["join_col"] = election_results["UNIQUE_ID"].str.upper()
matching = pd.read_csv("./raw-from-source/la_matching_csv.csv")
matching_dict = dict(zip(matching["join_col"], matching["UNIQUE_ID"].str.upper()))

In [6]:
statewide["join_col"] = statewide["join_col"].map(matching_dict).fillna(statewide["join_col"])

In [7]:
statewide.drop(["Parish","Precinct"], axis = 1, inplace = True)

In [8]:
joined = pd.merge(election_results,statewide, on = "join_col", how = "outer", indicator = True)

In [9]:
joined["_merge"].value_counts()

both          3800
right_only      64
left_only        5
Name: _merge, dtype: int64

In [10]:
joined[joined["_merge"]=="right_only"]["join_col"].unique()

array(['ACADIA-:-Parishwide', 'ALLEN-:-Parishwide',
       'ASCENSION-:-Parishwide', 'ASSUMPTION-:-Parishwide',
       'AVOYELLES-:-Parishwide', 'BEAUREGARD-:-Parishwide',
       'BIENVILLE-:-Parishwide', 'BOSSIER-:-Parishwide',
       'CADDO-:-Parishwide', 'CALCASIEU-:-Parishwide',
       'CALDWELL-:-Parishwide', 'CAMERON-:-Parishwide',
       'CATAHOULA-:-Parishwide', 'CLAIBORNE-:-Parishwide',
       'CONCORDIA-:-Parishwide', 'DE SOTO-:-Parishwide',
       'EAST BATON ROUGE-:-Parishwide', 'EAST CARROLL-:-Parishwide',
       'EAST FELICIANA-:-Parishwide', 'EVANGELINE-:-Parishwide',
       'FRANKLIN-:-Parishwide', 'GRANT-:-Parishwide',
       'IBERIA-:-Parishwide', 'IBERVILLE-:-Parishwide',
       'JACKSON-:-Parishwide', 'JEFFERSON-:-Parishwide',
       'JEFFERSON DAVIS-:-Parishwide', 'LAFAYETTE-:-Parishwide',
       'LAFOURCHE-:-Parishwide', 'LASALLE-:-Parishwide',
       'LINCOLN-:-Parishwide', 'LIVINGSTON-:-Parishwide',
       'MADISON-:-Parishwide', 'MOREHOUSE-:-Parishwide',
      

In [11]:
joined[joined["_merge"]=="left_only"]["join_col"].unique()

array(['IBERVILLE-:-00 22B', 'LINCOLN-:-12 03',
       'ST. JOHN THE BAPTIST-:-05 05', 'VERNON-:-05 02A',
       'WEST FELICIANA-:-00 02B'], dtype=object)

In [12]:
joined.columns

Index(['UNIQUE_ID', 'COUNTYFP', 'Parish', 'Precinct', 'G24A1NO', 'G24A1YES',
       'G24PREDHAR', 'G24PRELOLI', 'G24PREOCRU', 'G24PREOFRU', 'G24PREOKEN',
       'G24PREOPRE', 'G24PREOSON', 'G24PREOSTE', 'G24PREOTER', 'G24PREOWES',
       'G24PRERTRU', 'GCON01DMAN', 'GCON01NHYE', 'GCON01RARR', 'GCON01RSCA',
       'GCON01RSHA', 'GCON02DCAR', 'GCON02DDAV', 'GCON02RGRA', 'GCON02RLYN',
       'GCON02RPER', 'GCON03DGON', 'GCON03DSUM', 'GCON03RHIG', 'GCON03RJOH',
       'GCON04RJOH', 'GCON04RMOR', 'GCON05DVAL', 'GCON05RLET', 'GCON05RMEN',
       'GCON06DAND', 'GCON06DFIE', 'GCON06DJON', 'GCON06DWIL', 'GCON06RGUI',
       'join_col', 'TOTAL_Reg', 'WHITE_Reg', 'BLACK_Reg', 'OTHER_Reg',
       'TOTAL_Dem_Reg', 'WHITE_Dem_Reg', 'BLACK_Dem_Reg', 'OTHER_Dem_Reg',
       'TOTAL_Rep_Reg', 'WHITE_Rep_Reg', 'BLACK_Rep_Reg', 'OTHER_Rep_Reg',
       'TOTAL_Oth_Reg', 'WHITE_Oth_Reg', 'BLACK_Oth_Reg', 'OTHER_Oth_Reg',
       'TOTAL_Voted', 'WHITE_Voted', 'BLACK_Voted', 'OTHER_Voted',
       'TOTAL_Dem_Vot

In [13]:
if not os.path.exists("./la_2024_gen_prec_turnout/"):
    os.mkdir("./la_2024_gen_prec_turnout/")

In [14]:
joined = joined[joined["_merge"]=="both"]
joined.drop(["_merge","join_col"], axis = 1, inplace = True)

In [15]:
joined["COUNTYFP"] = joined["COUNTYFP"].astype(float).astype(int).astype(str).str.zfill(3)

In [16]:
clean_cols = [i for i in list(joined.columns) if "_Pct" not in i and i not in ["UNIQUE_ID","COUNTYFP","Parish","Precinct"]]

In [17]:
for col in clean_cols:
    joined[col] = joined[col].astype(int)

In [18]:
joined.to_csv("./la_2024_gen_prec_turnout/la_2024_gen_prec_turnout.csv", index = False)

In [19]:
import shutil

In [20]:
shutil.make_archive("./la_2024_gen_prec_turnout","zip","./la_2024_gen_prec_turnout")

'/Users/peterhorton/Documents/RDH/Support/Processing-Requests/LA_2024_Gen_Turnout/la_2024_gen_prec_turnout.zip'